# Interactive study (All)

This is deprecated since that study relies on our previous segmentation of the data, which was according to **all the sensors missing values**...

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import mpld3
from observation import Observation
from constants import *

plt.style.use('ggplot')
mpld3.enable_notebook()

PATH = "../../Data/GMPP_IRSDI/"

reactor_sites = [site+tranche for site in ["A","B","C","D","E","F","G","H"] for tranche in ["1","2"]] + ["B3","B4","F3","F4"]

suffixes = [
"DEB1-1","DEB1-2","DEB1-3","DEB1-4", # Débit de fuite au joint 1 (Gamme Large)
"DEB2-1","DEB2-2","DEB2-3","DEB2-4", # Débit de fuite au joint 1 (Gamme Étroite)
"DEB3-1","DEB3-2","DEB3-3","DEB3-4","DEB3-5", # Débit d'injection au joint
"PUI-",  # Puissance thermique moyenne
"PRE-",  # Pression
"TEM1-", # Température ligne d'injection aux joints (en * Celsius)     ### A rapprocher de DEB3
"TEM2-", # Température fuites joint 1
"TEM3-1","TEM3-2","TEM3-3","TEM3-4",# Température eau joint 1 - 051PO ### A rapprocher de DEB1 DEB2
"VIT-1","VIT-2","VIT-3","VIT-4"# Vitesse de rotation
] 

obs_dict = {}
intervals_dict = {}

for reactor_site in reactor_sites:
    print("Loading observation "+reactor_site)
    obs_dict[reactor_site] = Observation(PATH,reactor_site,suffixes)
    intervals_dict[reactor_site] = obs_dict[reactor_site].split_valid_intervals_df()

intervals = []
for key in intervals_dict.keys():
    intervals+=intervals_dict[key]

Loading observation A1
Loading observation A2
Loading observation B1


### Filtering and simple sorting ideas

In [ ]:
roll_var = lambda df: df[deb1].rolling(20).var().max(axis=0).max()
power_above_80 = lambda df: ((df[pui]<80).sum()==0).any()
jump = lambda df: df[deb1].diff().max().max()

### Trend detection

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split

def max_overall_slope(df):
    reg = LinearRegression()
    df = df[deb1]
    reg.fit(np.arange(len(df)).reshape(len(df),-1),df.values)
    return reg.coef_.max()*len(df)

def min_overall_slope(df):
    reg = LinearRegression()
    df = df[deb1]
    reg.fit(np.arange(len(df)).reshape(len(df),-1),df.values)
    return -reg.coef_.min()*len(df)

w_length = 30
def max_rolling_slope(df):
    reg = LinearRegression()
    df = df[deb1]
    l_reg = []
    for i in range(len(df)-w_length):
        fit = reg.fit(np.arange(w_length).reshape(w_length,-1), df.iloc[i:i+w_length].values) 
        l_reg += [(fit.coef_.max(),fit.coef_.argmax(),i)]
    return max(l_reg,key=lambda x: x[0])[0]

def min_rolling_slope(df):
    reg = LinearRegression()
    df = df[deb1]
    l_reg = []
    for i in range(len(df)-w_length):
        fit = reg.fit(np.arange(w_length).reshape(w_length,-1), df.iloc[i:i+w_length].values) 
        l_reg += [(fit.coef_.min(),fit.coef_.argmin(),i)]
    return -min(l_reg,key=lambda x: x[0])[0]

### Oscillation detection

In [ ]:
w_1 = 3; w_2 = 40
osc_filter = lambda df: len(df[deb1])>w_1+w_2
osc_var = lambda df: df[deb1].rolling(w_1).var().rolling(w_2).median().max(axis=0).max()

### Fourier ideas

In [ ]:
from cmath import *

strong_fourier_component = lambda df: np.max(np.abs(np.fft.fft(df[deb1],axis=0))[1:])
mean_fourier_component = lambda df: np.mean(np.abs(np.fft.fft(df[deb1],axis=0))[1:])
median_fourier_component = lambda df: np.median(np.abs(np.fft.fft(df[deb1],axis=0))[1:])

w_fourier = 100
fourier_filter = lambda df: len(df[deb1])>w_fourier
def rolling_fourier(df):
    df = df[deb1]
    current_fft = np.fft.fft(df.iloc[:w_fourier].values,axis=0)
    current_max = np.max(np.abs(current_fft)[2:int(w_fourier/2)])
    arange_op = np.exp(2j*pi*np.arange(w_fourier).reshape(-1,1)/w_fourier)
    for i in range(len(df)-w_fourier):
        current_fft = (current_fft - df.iloc[i].values + df.iloc[w_fourier+i].values)*arange_op
        current_max = max(current_max, np.max(np.abs(current_fft)[2:int(w_fourier/2)]))
    return current_max

### Interaction

In [ ]:
from interaction import Interaction

interaction = Interaction(intervals)
interaction.add_idx("Rolling variance",roll_var)
interaction.add_filter("Power above 80",power_above_80)
interaction.add_idx("Jump",jump)

interaction.add_idx("Increasing trend",max_overall_slope)
interaction.add_idx("Decreasing trend",min_overall_slope)

interaction.add_idx("Oscillation (var)",osc_var,osc_filter)
interaction.add_idx("Oscillation (fourier)",rolling_fourier,fourier_filter)
#interaction.add_idx("Mean Fourier Component",mean_fourier_component)
#interaction.add_idx("Strong Fourier Component",strong_fourier_component)
#interaction.add_idx("Median Fourier Component",median_fourier_component)

In [1]:
interaction.interact()

NameError: name 'interaction' is not defined